In [1]:
from imgphon import helmet 
from imgphon import imgphon as iph # TODO fix issue with detector and predictor
import os, subprocess, glob, re
import numpy as np
import audiolabel
import cv2
import dlib
from imutils.face_utils import FaceAligner
import matplotlib.pyplot as plt
%matplotlib inline

### Development area

In [ ]:
# tilt normalization on lips directly

In [ ]:
# tSNE
from sklearn.manifold import TSNE

n_tsne_comp = 2
X_embedded = TSNE(n_components=n_tsne_comp).fit_transform(analysis)

X_df = pd.DataFrame(X_embedded)
X_df.columns = ["D{}".format(d+1) for d in range(n_tsne_comp)]
X_df = X_df.assign(phone=pd.Series(md['uniquePhone']).values)
df = df.assign(phase=pd.Series(md['phase']).values)
X_df.head(5)
sns.lmplot(x="D1", y="D2", data=X_df, 
             hue="phone", fit_reg=False, size=5)

### Direct manipulations of video files (legacy)

In [ ]:
# extract and display a single landmark-annotated video frame
video = 'SN12_AA032001.MXF'
time = '00:00:10.900'

frame = get_video_frame(video,time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

In [ ]:
# extract a frame corresponding to the desired timepoint of an audio file (here, from a TextGrid) 
# and extracts a polygon of the lip shape
babb = 'vowels/P1.MXF'
babb_tg = os.path.splitext(babb)[0] + ".TextGrid"
pm = audiolabel.LabelManager(from_file=babb_tg, from_type="praat")
v = pm.tier('phone').search(vre)[0] # should only be one match per file; TODO check while running
midpoint= v.center
ffmpeg_time = str(round(midpoint,3))
frame = get_video_frame(babb,ffmpeg_time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

image = lip_mask(frame,shape)
plt.imshow(crop_center(image))

In [ ]:
# clip a short stretch of video from the longer videos typical of ultrasound acquisitions
exp_movie = '12-video/SN12_AA032001.MXF'
exp_snippet = '12-video/SN12_AA032001_short.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

# get a snippet of the MXF
snippet_args = ['ffmpeg', '-ss', '00:00:30', 
                '-i', exp_movie, 
                "-t", "00:00:08", 
                "-vcodec", "copy", 
                "-acodec", "copy", 
                exp_snippet]
subprocess.check_call(snippet_args)

In [ ]:
exp_movie = 'vowels/POO1.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

subprocess.check_call(['ffmpeg','-loglevel','8','-i',exp_snippet,'-vf','fps=20','img_%05d_f.bmp'])
movie = cv2.VideoCapture('img_%05d_f.bmp')

frame_num = 1

while(movie.isOpened()):
    ret, frame = movie.read()
    if (ret==False):   # the file is finished
        break

    # detect face region and draw landmarks on the image.
    shape = detect_landmarks(frame)
    out_image = draw_landmarks(frame,shape)
    
    # write the image to a .bmp file, with zero-padding to ensure the frames are input 
    cv2.imwrite('{0:05d}g.bmp'.format(frame_num), out_image)
    frame_num += 1
        
# cleanup
cv2.destroyAllWindows()

# make a gif (a bit slowed down by default) from the output bmps
subprocess.check_call(['convert', '*g.bmp', basename+'.gif'])

# remove the input bmps
for bmp in glob.glob("*g.bmp"):
    os.remove(bmp)
for bmp in glob.glob("*f.bmp"):
    os.remove(bmp)